In [92]:
import numpy as np
import pandas as pd
import networkx as nx # https://networkx.org/documentation/stable/index.html

In [37]:
# read in meta data
metadata = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", nrows=1, header=None).iloc[0]
s = metadata.iloc[0]
n = metadata.iloc[1]
m = metadata.iloc[2]
l = metadata.iloc[3]

In [38]:
df = pd.read_csv("data\inst_tuning\heur040_n_300_m_13358.txt", sep=" ", skiprows=1, names = ["n1", "n2", "e", "w"])
df

I am not sure if it is really necessary to create the matrices A and w, if we use the networkx library. w might be usefull, but maybe A is not needed. I just tried it out before discovering the networkx package...
The code like this gives an (n-1)x(n-1) data frame where only the upper triangel is filled. If we want to represent it differently (e.g. symmetric matrix, boolean matrix, 2Dnumpy array, real upper triangular matrix etc) more transformation is needed.

I think for w it makes sense to use the matrix, because we only need it for calculating f. We have to be carefull with index, because the matrix has dimension (n-1)x(n-1) because the diagonal of the n x n matrix would just be 0 (connecting each node with itself costs nothing). Better to work with the row and column index then...

In [52]:
# create adjacency matrix as data frame of upper triangular matrix
A = df.pivot(index='n1', columns='n2', values='e') #gives upper triangular data frame
A

n2,2,3,4,5,6,7,8,9,10,11,...,291,292,293,294,295,296,297,298,299,300
n1,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [49]:
# create weight matrix as data frame of upper triangular matrix
w = df.pivot(index='n1', columns='n2', values='w') #gives upper triangular data frame
w

n2,2,3,4,5,6,7,8,9,10,11,...,291,292,293,294,295,296,297,298,299,300
n1,,,,,,,,,,,,,,,,,,,,,
1,1.0,2.0,6.0,1.0,3.0,1.0,6.0,7.0,5.0,2.0,...,8.0,3.0,6.0,4.0,1.0,4.0,1.0,1.0,2.0,1.0
2,NaN,5.0,2.0,1.0,5.0,7.0,6.0,5.0,2.0,4.0,...,3.0,7.0,1.0,1.0,4.0,1.0,1.0,2.0,6.0,2.0
3,NaN,NaN,2.0,3.0,3.0,1.0,3.0,5.0,1.0,6.0,...,4.0,1.0,3.0,1.0,1.0,4.0,2.0,2.0,3.0,2.0
4,NaN,NaN,NaN,1.0,1.0,2.0,5.0,1.0,3.0,2.0,...,1.0,2.0,2.0,3.0,4.0,2.0,3.0,3.0,5.0,2.0
5,NaN,NaN,NaN,NaN,2.0,3.0,3.0,3.0,2.0,5.0,...,1.0,1.0,1.0,4.0,2.0,2.0,2.0,4.0,6.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0,4.0,2.0
296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,3.0
297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,1.0


In [96]:
w.loc[2,3] # weight of edge connecting node 2 and 3

5.0

In [97]:
np.nansum(A.loc[4,])+np.nansum(A.loc[:,4]) # this should give us d(node4), as it is an upper triangular symmetric matrix. 
# But I don't know if we need this, as it is probably implemented in networkx.

43.0

In [93]:
# try out how networkx works
# create empty graph
g = nx.Graph()

# get currently used edges (plus their weight, if we want)
current_e = df.loc[df["e"]==1][["n1", "n2", "w"]].to_numpy()
# fill graph by loading it with edges (with weights)
g.add_weighted_edges_from(current_e, weight = "w")


In [94]:
# we can find out which nodes are currently connected
# should give us a list of nodes that are connected
d = list(nx.connected_components(g))
# d contains disconnected subgraphs
# d[0] contains the biggest subgraph
d

[{1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  

In [121]:
# try this function with a completely disconnected graph to see the difference
g_disconnected = nx.Graph()
g_disconnected.add_nodes_from(range(1, 20))
d1 = list(nx.connected_components(g_disconnected))
d1

[{1},
 {2},
 {3},
 {4},
 {5},
 {6},
 {7},
 {8},
 {9},
 {10},
 {11},
 {12},
 {13},
 {14},
 {15},
 {16},
 {17},
 {18},
 {19}]

In [120]:
for i in d1[0]:
    rem = df.loc[((df["n1"]==i) | (df["n2"]==i)) & (df["e"]==1)][["w"]].sum()
    print(rem)

w    431
dtype: int64
